# Identificar fraude no email da Enron 

### Primeira Parte: Fundamentação Teórica e Explicação do Código

O objetivo desse projeto é determinar se um funcionário é ou não um funcionário de interesse (POI). Um funcionário de interesse é um funcionário que participou do escândalo da empresa Enron. 

Para isso iremos usar diversos métodos que foram aprendidos no módulo de *Machine Learning*, mais especificamente usaremos o conjunto de métodos **sklearn**, que contém inúmeras funcionalidades para esse aprendizado :)

Meu código final está no arquivo *poi_id.py* que deve ser executado primeiro para exportar os conjuntos de dados e em seguida deve-se executar o *tester.py*.

Machine Learning foi a solução ideal para a resolução desse problema pois para sabermos se um funcionário está ou não envolvido no escândalo, nós precisamos que o nosso código saiba o que é isso e como classificar isso.
Existem diversos tipos de ML tais como recomendação (que Netflix e Amazon usam e abusam), detecção de anomalias, prevenção de fraude, agrupamento e diversos outros.

Para a resolução desse problema eu testei **4 algoritmos** que aprendi ao longo do módulo:
- **Decision Tree:** Esse algoritmo pode ser comparado ao *CASE WHEN... THEN.. ELSE... END". Basicamente nós aplicamos certas regras nos dados e dependendo de cada valor, um decisão é tomada. Esse algoritmo é bastante usado em pesquisas operacionais.
- **Random Forest:** É um *ensemble learning* supervisionado, que é comumente usado em problemas de classificação, detecção de anomalias e redes neurais.
- **SDG:** Esse método é usado em otimização e tem como objetivo encontrar o mínimo local de uma função.
- **NaiveBayes:** É bastante usado em *text learning* 

Além dos 4 métodos apresentados acima, existem alguns cuidados, ou melhor, tratativas que devem ser aplicadas no dataset antes de treinar o algoritmo.

Um ponto bastante importante é a **remoção de outliers**. Classificamos um valor como outlier quando ele está muito discrepante do restante dos valores. Existem algumas técnicas para a detecção/exclusão, uma bastante conhecida e que aprendi aqui no Nanodegree é o [Interquartile Range (IQR)](https://en.wikipedia.org/wiki/Interquartile_range).

Nesse projeto tive que remover um valor do conjunto de dados. 
   - `data_dict.pop('TOTAL')`: *TOTAL* não é uma pessoa e isso estava poluindo a análise.
   
Outro ponto de limpeza dos dados que precisei fazer foi realizar um replace em todos os registros cujos valores eram **NaN**. Isso poderia afetar bastante as análises, então decidi apenas substituir por **0**:
   - `final = df[cols].copy().applymap(lambda x: 0  if x == 'NaN' else x)`
   
Agora sobre as **features** que foram analizadas aqui, utilizei o algoritmo [SelectPercentile](http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectPercentile.html) e [FeatureSelection](http://scikit-learn.org/stable/modules/feature_selection.html) para saber quais features mais eram relevantes para a descoberta da nossa pergunta.
O primeiro foi para pegar um percentual do conjunto de dados e o segundo foi utilizado para a seleção das features de fato :)

Tendo nossas features selecionadas e armazenadas em *features_list*, executei o seguinte código para fazer a divisão em **labels** e **features** para que possamos começar a testar os algoritmos de ML!

```
data = featureFormat(my_dataset, features_list, sort_keys = True)
labels, features = targetFeatureSplit(data)
```

Esse conjunto *featureFormat* foi provido pela Udacity e pode ser encontrado [aqui](https://github.com/udacity/ud120-projects/blob/master/tools/feature_format.py).

Por fim, utilizei o método de [cross_validation](http://scikit-learn.org/stable/modules/cross_validation.html) para realizarmos a divisão de nosso dataset em dois, um para teste e outro para treinamento. 
Realizar essa divisão é essencial, pois ele nos ajuda a evitarmos bastante a ocorrência de *overfitting*, porém alguns cuidados devem ser tomados, como por exemplo, a divisão precisa ser randômica, para que não treinamos nosso algoritmo e deixe ele com um viés mais forte. 
Para isso, dentro desse método, existe o [test_train_split](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html#sklearn.model_selection.train_test_split) que faz essa divisão e já leva em conta esse cuidado.

Aqui eu fiz a divisão clássica de 30% para teste e 70% para treino.

Na segunda parte entrarei um pouco mais detalhado no código e em alguns testes e tunings que fiz com alguns parâmetros para que fosse possível obter a melhor resposta :)

### Segunda Parte: Código executado

In [1]:
import sys
import os
sys.path.append("../ud120-projects/")

In [4]:
## DecisionTreeeClassifier
run tester.py

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')
	Accuracy: 0.76691	Precision: 0.35142	Recall: 0.33350	F1: 0.34223	F2: 0.33694
	Total predictions: 11000	True positives:  667	False positives: 1231	False negatives: 1333	True negatives: 7769



In [7]:
## DecisionTreeeClassifier
run tester.py

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)
	Accuracy: 0.82473	Precision: 0.54380	Recall: 0.22350	F1: 0.31680	F2: 0.25334
	Total predictions: 11000	True positives:  447	False positives:  375	False negatives: 1553	True negatives: 8625



In [10]:
## SDGClassifier
run tester.py

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', n_iter=5, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=42, shuffle=True, verbose=0,
       warm_start=False)
	Accuracy: 0.58045	Precision: 0.16293	Recall: 0.31600	F1: 0.21500	F2: 0.26602
	Total predictions: 11000	True positives:  632	False positives: 3247	False negatives: 1368	True negatives: 5753



In [11]:
## NaiveBayesClassifier
run tester.py

GaussianNB(priors=None)
	Accuracy: 0.83255	Precision: 0.56280	Recall: 0.35400	F1: 0.43462	F2: 0.38237
	Total predictions: 11000	True positives:  708	False positives:  550	False negatives: 1292	True negatives: 8450



Como pode-se perceber, aqui não precisei tunar o modelo para conseguir o mínimo requerido para o projeto (0.3 para ambas as métricas).

Quando calculamos a métrica de precisão, queremos saber qual é a razão entre os eventos que previmos corretamente com todos os outros que eram corretos, porém nosso algoritmo ou fez a previsão correta, ou classificou como negativo. Já recall é quando queremos saber a taxa de verdadeiros positivos no problema.

Um tratamento que fiz foi selecionar o melhor *percentile* para o conjunto de dados. O que esse método faz é selecionar as **features de acordo com o seu score**.

Seguem abaixo alguns testes que fiz até chegar no melhor modelo :)

Eu usei o NaiveBayes como teste para achar o melhor Percentile :)

Meu conjunto de features final ficou sendo o seguinte:

### Terceira Parte: Possíveis problemas com o conjunto de dados

Aqui irei pontuar em tópicos alguns problemas que encontrei nesse conjunto de dados:
   - Inconsistência nos e-mails
   - Diferentes conjuntos de dados podem introduzir diferentes bias e erros
   - POI pode não estar no conjunto de dados